# SDP Product Equations

This notebook generates an overview of the formulas used to calculate product properties by the parametric model.

In [ ]:
from implementation import PipelineConfig
from parameter_definitions import *
from api import SkaPythonAPI as api
from IPython.display import display, Math, Latex, HTML
from sympy import latex, Lambda

# Configuration to show formulas for
cfg = PipelineConfig(telescope=Telescopes.SKA1_Mid,
                     pipeline=Pipelines.ICAL,
                     band=Bands.Mid1)
display(HTML("<h3>Equations for %s:</h3>" % cfg.pipeline))

# Calculate values and formulas
expr = 'Rflop'
unit = 'PFLOP/s'
mult = Constants.peta
values = api.eval_expression_default_products(cfg, expression=expr)
formulas = api.eval_products_symbolic(cfg, expression=expr)

# Show equations for products
maths = ""
for product in sorted(values.keys()):
    maths += (r'%s_{%s} &= %s \\ &= %f\,\text{%s} \\ \\' %
        (expr, product,
         latex(formulas.get(product, 0)),
         values.get(product, 0) / mult, unit))
maths = (r"\begin{align}%s %s &= %f\,\text{%s}\end{align}" % \
    (maths, expr, sum(values.values()) / mult, unit))
display(Math(maths))

In [ ]:
# Look up free symbols to show helpers
display(HTML("<h4>Helper equations:</h4>")); maths = ""
free_syms = api.collect_free_symbols(formulas.values())
sym_defs = api.eval_symbols(cfg, free_syms, symbolify='all')
for sym in sorted(sym_defs):
    if isinstance(sym_defs[sym], BLDep):
        maths += r'%s(%s) &= %s \\' % (
            latex(Symbol(sym)),
            sym_defs[sym].b,
            latex(sym_defs[sym].term))
    else:
        maths += r'%s &= %s \\' % (latex(Symbol(sym)), latex(sym_defs[sym]))
display(Math(r"\begin{align}%s\end{align}" % maths))

In [ ]:
# And finally all remaining constants
display(HTML("<h4>Constants:</h4>")); maths = ""
free_syms2 = api.collect_free_symbols(sym_defs.values())
consts = set(free_syms).union(free_syms2)
const_defs = api.eval_symbols(cfg, consts)
for sym in sorted(const_defs):
    if not str(sym) in sym_defs:
        maths += r'%s &= %s \\' % (latex(Symbol(sym)), latex(const_defs[sym]))
display(Math(r"\begin{align}%s\end{align}" % maths))